# 2022-49 Mode Choice and SB743 VMT QC

In [2]:
import pandas as pd
import sqlalchemy as sql

ddam = sql.create_engine('mssql+pymssql://DDAMWSQL16/demographic_warehouse')

## Mode Choice Report

### Study Area Trips section

In [2]:
# The list of MGRAs that define the downtown area
mgra = pd.read_csv("T:/ABM/SB/Enhancement_ABM2+/data/PowerBI/ModeChoiceReport/service_bureau_mode_choice_reports/QA/cicpa_downtown_mgra.csv").values
mgra = tuple([str(y) for z in mgra for y in z])

# The scenario_id to use
SCENARIO_ID = 100

# geography_set_id = 4 corresponds to "zone split for Seaport Village land use converter test 
# 6.1.2022". This value is used because in [abm_14_3_0].[dimension].[scenario], scenario_id=100 
# corresponds to geography_set_id = 4
GEOGRAPHY_SET_ID = 4

In [3]:
# Double check that the MGRAs which define the downtown area are correct
downtown_mgras = pd.read_sql(f"""
    SELECT mgra_13
    FROM [abm_14_3_0].[dimension].[geography]
    WHERE cicpa_2016_name='Downtown'
        AND mgra_13 != 'Not Applicable'
        AND geography_set_id = {GEOGRAPHY_SET_ID}
""", con=ddam.connect())
mgra == tuple(downtown_mgras["mgra_13"])

True

In [4]:
# Get all the person trips from SQL filtered by scenario_id, the list of MGRAs in the jurisdiction,
# and the correct geography_set_id
person_trips = pd.read_sql(f"""
    SELECT 
        geo_o.mgra_13 AS origin_mgra, 
        geo_d.mgra_13 AS dest_mgra, 
        trip.*
    FROM [abm_14_3_0].[fact].[person_trip] AS trip
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_o ON
        trip.geography_trip_origin_id = geo_o.geography_id
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_d ON
        trip.geography_trip_destination_id = geo_d.geography_id
    WHERE scenario_id={SCENARIO_ID}
        AND (geo_o.mgra_13 IN {mgra} OR geo_d.mgra_13 IN {mgra})
        AND geo_o.geography_set_id = {GEOGRAPHY_SET_ID}
        AND geo_d.geography_set_id = {GEOGRAPHY_SET_ID}
    """, con=ddam.connect())

In [ ]:
# Intra-Zonal = trip starts and ends in the same MGRA
intra_zonal = person_trips[person_trips["origin_mgra"] == person_trips["dest_mgra"]].copy(deep=True)

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Intra-Zonal values")
print(f"{'Pre filter trips:' : <20}", intra_zonal.shape[0])
intra_zonal = intra_zonal[intra_zonal["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", intra_zonal.shape[0])

Intra-Zonal values
Pre filter trips:    20170
Post filter trips:   18392


In [ ]:
# Inter-Zonal = trip starts and ends in the same jurisdiction, but not the same MGRA
inter_zonal = person_trips[person_trips["origin_mgra"] != person_trips["dest_mgra"]].copy(deep=True)
inter_zonal = inter_zonal[inter_zonal["origin_mgra"].isin(mgra) & inter_zonal["dest_mgra"].isin(mgra)]

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Inter-Zonal values")
print(f"{'Pre filter trips:' : <20}", inter_zonal.shape[0])
inter_zonal = inter_zonal[inter_zonal["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", inter_zonal.shape[0])

Inter-Zonal values
Pre filter trips:    342230
Post filter trips:   307715


In [ ]:
# Inter-Study Area = trip starts/ends in the jurisdiction, but ends/starts outside the jurisdiction
inter_study = person_trips.copy(deep=True)
inter_study = inter_study[
    (inter_study["origin_mgra"].isin(mgra) & ~inter_study["dest_mgra"].isin(mgra)) |
    (~inter_study["origin_mgra"].isin(mgra) & inter_study["dest_mgra"].isin(mgra))]

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Inter-Study Area values")
print(f"{'Pre filter trips:' : <20}", inter_study.shape[0])
inter_study = inter_study[inter_study["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", inter_study.shape[0])

Inter-Study Area values
Pre filter trips:    700524
Post filter trips:   616203


## VMT

### VMT Per Employee by Geography

In [3]:
# The list of TAZs that define the downtown area
taz = pd.read_csv("T:/ABM/SB/Enhancement_ABM2+/data/PowerBI/SB743_VMT/QA/sv_luct_zs_taz.csv").values
taz = tuple([str(y) for z in taz for y in z])

# The scenario_id to use
SCENARIO_ID = 100

# geography_set_id = 4 corresponds to "zone split for Seaport Village land use converter test 
# 6.1.2022". This value is used because in [abm_14_3_0].[dimension].[scenario], scenario_id=100 
# corresponds to geography_set_id = 4
GEOGRAPHY_SET_ID = 4

In [4]:
taz

('4070', '4123', '4997', '4998', '4999')

In [ ]:
# # Double check that the TAZs which define the downtown area are correct
# downtown_tazs = pd.read_sql(f"""
#     SELECT *
#     FROM [abm_14_3_0].[dimension].[geography]
#     WHERE geography_set_id = 4
#         AND jurisdiction_2016_name = 'San Diego'
#         AND cicpa_2016_name = 'Downtown'
# """, con=ddam.connect())
# taz == tuple(downtown_tazs["mgra_13"])

As far as I can tell, there is no way to distinguish the TAZs in the file using the columns of the table [abm_14_3_0].[dimension].[geography]. This makes errors harder to debug as we would be looking at a list of TAZs persumably in some other software like ArcGIS rather than keeping all the data in SQL.

In [ ]:
# Get all the person trips from SQL filtered by scenario_id, the list of MGRAs in the jurisdiction,
# and the correct geography_set_id
person_trips = pd.read_sql(f"""
    SELECT 
        geo_o.taz_13 AS origin_taz, 
        geo_d.taz_13 AS dest_taz, 
        trip.*,
        tour_purpose.purpose_tour_description, 
        tour_purpose.purpose_tour_aggregate_description
    FROM [abm_14_3_0].[fact].[person_trip] AS trip
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_o ON
        trip.geography_trip_origin_id = geo_o.geography_id AND
        geo_o.geography_set_id = {GEOGRAPHY_SET_ID}
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_d ON
        trip.geography_trip_destination_id = geo_d.geography_id AND
        geo_d.geography_set_id = {GEOGRAPHY_SET_ID}
    LEFT JOIN [abm_14_3_0].[dimension].[tour] AS tours ON
        trip.tour_id = tours.tour_id AND
        trip.scenario_id = tours.scenario_id
    LEFT JOIN [abm_14_3_0].[dimension].[purpose_tour] AS tour_purpose ON
        tours.purpose_tour_id = tour_purpose.purpose_tour_id
    LEFT JOIN [abm_14_3_0].[dimension].[person] AS person ON
        trip.person_id = person.person_id AND
        trip.scenario_id = person.scenario_id
    WHERE trip.scenario_id={SCENARIO_ID} AND 
        (geo_o.taz_13 IN {taz} OR geo_d.taz_13 IN {taz}) AND 
        trip.person_id > 0 AND 
        person.work_segment != 'Not Applicable'
    """, con=ddam.connect())

NameError: name 'pd' is not defined

In [ ]:
person_trips

,origin_taz,dest_taz,scenario_id,person_trip_id,model_trip_id,abm_trip_id,tour_id,person_id,household_id,mode_trip_id,...,time_mt,distance_mt,cost_fare_mt,time_bike,distance_bike,time_total,distance_total,cost_total,purpose_tour_description,purpose_tour_aggregate_description
0,4019,4998,100,19815153,1,9732122,1738800,2581920,1034658,4,...,0.0,0.0,0.0,0.0,0.0,11.826000,0.591300,0.00,Shop,Shop
1,4019,4998,100,19815507,1,9732478,1739792,2582025,1034717,4,...,0.0,0.0,0.0,0.0,0.0,11.826000,0.591300,0.00,Shop,Shop
2,4019,4998,100,19815539,1,9732510,1739795,2582034,1034721,4,...,0.0,0.0,0.0,0.0,0.0,11.826000,0.591300,0.00,Shop,Shop
3,4019,4998,100,19815618,1,9732589,1740648,2582056,1034731,4,...,0.0,0.0,0.0,0.0,0.0,11.826000,0.591300,0.00,Shop,Shop
4,4019,4998,100,19815681,1,9732639,1740651,2582069,1034741,4,...,0.0,0.0,0.0,0.0,0.0,11.826000,0.591300,0.00,Shop,Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97769,4070,4879,100,23108478,1,13025440,3367921,3459092,1338086,9,...,0.0,0.0,0.0,0.0,0.0,80.385107,16.436830,2.03,Work,Work
97770,4070,4392,100,23359163,1,13276120,1015732,3525965,1360450,9,...,0.0,0.0,0.0,0.0,0.0,67.327152,8.743359,2.03,Shop,Shop
97771,4860,4070,100,23031448,1,12948407,2787169,3437734,1331444,9,...,0.0,0.0,0.0,0.0,0.0,89.958221,18.172874,2.03,Work,Work
97772,4879,4070,100,23108475,1,13025437,3367921,3459092,1338086,9,...,0.0,0.0,0.0,0.0,0.0,82.806028,16.767543,2.03,Work,Work


In [ ]:
person_trips["work_segment"]

KeyError: 'work_segment'